# 📊 Proyecto: Análisis y visualización de la calidad del aire: una exploración de contaminantes atmosféricos y su relación con el PM2.5
**Curso:** Samsung Innovation Campus – Módulo de Python (Ecuador 2025)  
**Seccion:** EC03  

---

## 🧩 Módulo 1: Adquisición de Datos

En esta sección se carga el dataset, se explora su estructura general y se validan los rangos de los contaminantes.

El dataset contiene mediciones de diferentes gases y partículas contaminantes en el aire, registradas con fecha y hora.


In [ ]:
import pandas as pd
import numpy as np
import warnings
import src.data_acquisition as da

warnings.filterwarnings('ignore')